<a href="https://colab.research.google.com/github/ManasChandan/deep-learning-series/blob/master/Deep_Learning_3_CNN_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Spatial Adjancey information for CNN
# Problems with directly using the image into a 100 neuron dense layer.
# Too much parameters.
# Translational Invariance for deep learning (ex a vertical line)
# What is a convolutional filter and how it primarily solves the above issues ?
# What is exactly applying the filter to the image matrix.
# Nuances while applying - Padding and Strides.
# How these filters get hold of the edges
# Now aht are convolution layer and how multiple layer stack into them.
# How exactly a rgb photo tensor is applied a convolution filter like increasing its dimensions.
# How the filter can be understood as neurons which when pass through network and they learn.
# Polling layers
# What is the intuition behind pooling layers how exactly do they work ?
# What is data augmentation ?
# Adveserial risks for transfer learning ?
# Upsampling layer and billinear interpollation and Nearest interpollatin
# Batch Normalization
# GroupAveragePooling and simple pooling difference
# A bit on transfer learning, how to do featre extraction at once, internal preprocess function.

# Great question - How emails are classified ?

In [2]:
# Let us train on a colured image dataset our simple conv network and see the accuracy The CIFAR 100

from tensorflow import keras
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
keras.utils.set_random_seed(42)

In [10]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data(label_mode="fine")

In [11]:
x_train.shape

# This is 50,000 image training data with 32,32 pixels coloured image.

(50000, 32, 32, 3)

In [6]:
# Lets directly start building a nn

inputs = keras.layers.Input(shape=x_train.shape[1:])

x = keras.layers.Rescaling(1./255)(inputs)

x = keras.layers.Conv2D(
    filters = 32,
    kernel_size = (2,2),
    activation = "relu"
)(x)

x = keras.layers.MaxPooling2D(
    pool_size = (2,2)
)(x)

x = keras.layers.Conv2D(
    filters = 64,
    kernel_size = (3,3),
    activation = "relu"
)(x)

x = keras.layers.MaxPooling2D(
    pool_size = (2,2)
)(x)

x = keras.layers.Flatten()(x)

x = keras.layers.Dense(128, activation="relu")(x)

x = keras.layers.Dense(256, activation="relu")(x)

x = keras.layers.Dense(128, activation="relu")(x)

x = keras.layers.Dense(len(np.unique(y_train)), activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=x)

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_1 (Rescaling)         │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 31, 31, 32)     │           416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 100)            │        12,900 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 392,772 (1.50 MB)

 Trainable params: 392,772 (1.50 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(),
    optimizer = keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [8]:
history = model.fit(
    x_train, y_train,
    epochs = 20,
    batch_size = 32,
    validation_split=0.2,
    verbose=1
)

Epoch 1/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.0410 - loss: 4.2825 - val_accuracy: 0.1371 - val_loss: 3.6293
Epoch 2/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.1654 - loss: 3.4490 - val_accuracy: 0.2133 - val_loss: 3.1923
Epoch 3/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.2361 - loss: 3.0602 - val_accuracy: 0.2607 - val_loss: 2.9830
Epoch 4/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.2879 - loss: 2.8164 - val_accuracy: 0.2855 - val_loss: 2.8731
Epoch 5/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.3274 - loss: 2.6272 - val_accuracy: 0.2984 - val_loss: 2.8111
Epoch 6/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.3592 - loss: 2.4750 - val_accuracy: 0.3068 - val_loss: 2.7917
Epoch 7/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.3880 - loss: 2.3349 - val_accuracy: 0.3088 - val_loss: 2.8329
Epoch 8/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.4128 - loss: 2.2047 

In [20]:
# Clearly the verbose is showing the simple model is not working ....
# Lets apply transfoer learning from ResNet.

# But a problem, Resnet needs 224,224,3 -- We have 32,32,3
# Solution UpSampling Layer.

upscaling_layer = keras.layers.UpSampling2D(size=(7,7), interpolation="bilinear")

In [19]:
# Resnet model

resnet = keras.applications.ResNet152V2(
    include_top = False,
    weights = "imagenet",
    input_shape = (224,224,3)
)

resnet.trainable = False
for layer in resnet.layers:
    layer.trainable = False

In [21]:
# The model

inputs = keras.layers.Input(shape=x_train.shape[1:])

x = upscaling_layer(inputs)

x = keras.layers.Rescaling(1./255)(x)

x = resnet(x)

x = keras.layers.Flatten()(x)

x = keras.layers.Dense(128, activation="relu")(x)

x = keras.layers.Dense(256, activation="relu")(x)

x = keras.layers.Dense(128, activation="relu")(x)

x = keras.layers.Dense(len(np.unique(y_train)), activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=x)

model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(),
    optimizer = keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_2 (UpSampling2D)  │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_5 (Rescaling)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet152v2 (Functional)        │ (None, 7, 7, 2048)     │    58,331,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 128)            │    12,845,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 100)            │        12,900 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 71,255,652 (271.82 MB)

 Trainable params: 12,924,004 (49.30 MB)

 Non-trainable params: 58,331,648 (222.52 MB)

In [22]:
for layer in model.layers:
    print(f"Layer: {layer.name}, Trainable: {layer.trainable}")

Layer: input_layer_8, Trainable: True
Layer: up_sampling2d_2, Trainable: True
Layer: rescaling_5, Trainable: True
Layer: resnet152v2, Trainable: False
Layer: flatten_3, Trainable: True
Layer: dense_13, Trainable: True
Layer: dense_14, Trainable: True
Layer: dense_15, Trainable: True
Layer: dense_16, Trainable: True


In [23]:

history_tl = model.fit(
    x_train, y_train,
    epochs = 50,
    batch_size = 32,
    validation_split=0.2,
    verbose=1
)

Epoch 1/50
 520/1250 ━━━━━━━━━━━━━━━━━━━━ 2:09 178ms/step - accuracy: 0.2554 - loss: 3.3425

KeyboardInterrupt: 

In [ ]:
# The epoch run-time is too high. On research I get thet the re-scaling + Upsampling and forward pass is being done each time. so let me
# pre-calculate them and save for speed.

extraction_input = keras.layers.Input(shape=x_train.shape[1:])
x = upscaling_layer(extraction_input)
x = keras.layers.Rescaling(1./255)(x)
x = resnet(x)
x = keras.layers.Flatten()(x)

feature_extractor = keras.Model(inputs=extraction_input, outputs=x)

print("Step 1: Extracting features (running ResNet once)...")
train_features = feature_extractor.predict(x_train, batch_size=32, verbose=1)
test_features = feature_extractor.predict(x_test, batch_size=32, verbose=1)

num_classes = len(np.unique(y_train))
fast_input = keras.layers.Input(shape=(train_features.shape[1],))

x = keras.layers.Dense(128, activation="relu")(fast_input)
x = keras.layers.Dense(256, activation="relu")(x)
x = keras.layers.Dense(128, activation="relu")(x)
outputs = keras.layers.Dense(num_classes, activation="softmax")(x)

final_model = keras.Model(inputs=fast_input, outputs=outputs)

final_model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

print("\nStep 2: Training the Dense layers...")
hsitory_tl = final_model.fit(
    train_features, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(test_features, y_test)
)

Step 1: Extracting features (running ResNet once)...
 691/1563 ━━━━━━━━━━━━━━━━━━━━ 2:39 183ms/step

In [13]:
# The Upsamplig - killed my Free Collab GPU :)

# 1. Define the Resize + Feature Extractor model
# pooling='avg' turns the 7x7x2048 output into a simple 2048 vector
resnet_base = keras.applications.ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3),
    pooling='avg'
)
resnet_base.trainable = False

# We group the resizing and the base model together
feature_extractor = keras.Sequential([
    keras.layers.Input(shape=x_train.shape[1:]),  # Original size (e.g., 32, 32, 3)
    keras.layers.Resizing(224, 224),              # Resize to ResNet standard
    keras.layers.Lambda(keras.applications.resnet50.preprocess_input),
    resnet_base
])

# 2. Extract Features (Pre-calculating)
# We use batch_size=32 to keep the GPU/RAM from choking
print("Extracting features... This is the one-time 'Heavy' step.")
train_features = feature_extractor.predict(x_train, batch_size=32, verbose=1)
test_features = feature_extractor.predict(x_test, batch_size=32, verbose=1)

# 3. Define the Final Lightweight Model
num_classes = len(np.unique(y_train))
final_model = keras.Sequential([
    keras.layers.Input(shape=(train_features.shape[1],)),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(256, activation="relu"),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(num_classes, activation="softmax")
])

# 4. Compile and Train
final_model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

print("\nTraining the Dense layers (This should be instant)...")
history = final_model.fit(
    train_features, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(test_features, y_test)
)

Extracting features... This is the one-time 'Heavy' step.
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 135s 82ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 25s 79ms/step

Training the Dense layers (This should be instant)...
Epoch 1/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.4236 - loss: 2.2393 - val_accuracy: 0.6173 - val_loss: 1.3364
Epoch 2/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.6611 - loss: 1.1675 - val_accuracy: 0.6424 - val_loss: 1.2883
Epoch 3/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7146 - loss: 0.9684 - val_accuracy: 0.6588 - val_loss: 1.2874
Epoch 4/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7485 - loss: 0.8357 - val_accuracy: 0.6496 - val_loss: 1.3620
Epoch 5/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.7730 - loss: 0.7315 - val_accuracy: 0.6537 - val_loss: 1.4069
Epoch 6/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7950 - loss: 0.6522 - val_accuracy: 0.6513 - val_loss: 1.5064
Epoch 7/20
1563/15

* **Frozen "Generic" Brain:** By keeping `trainable = False`, you are using features optimized for ImageNet (high-res photos of dogs, cars, etc.). These don't translate perfectly to the 100 specific, nuanced classes of CIFAR-100 (like different types of trees or insects) without fine-tuning.
* **Low "Information Density":** Upsampling a  image to  doesn't add new detail; it just creates a blurry version of the original. The ResNet filters, which expect sharp edges at high resolution, struggle to extract deep meaning from these "pixelated" artifacts.
* **Data Scarcity per Class:** CIFAR-100 only provides **500 training images per category**. Without heavy **Data Augmentation** (flips, rotations, crops), the model quickly memorizes the training set instead of learning general patterns.
* **Internal Covariate Shift:** Your Dense layers were trained on the output of a frozen ResNet. If you don't use **Batch Normalization** or a very specific learning rate during a "fine-tuning" phase, the model can't adjust the backbone to better separate the 100 competing classes.
* **The "Bottleneck" Loss:** ResNet-50's `GlobalAveragePooling` compresses the image into a 2048-dimensional vector. For 100 classes, this is a tight squeeze. If the ResNet isn't fine-tuned to prioritize the *right* features for CIFAR, your Dense layers are basically trying to find a needle in a haystack of irrelevant ImageNet features.